# CleanUp


## Prerequisites

Select the `Python 3 (Data Science)` kernel in SageMaker Studio.

In [8]:
import boto3
import sagemaker

region = boto3.Session().region_name
print(f"Region: {region}")

boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client("sagemaker")
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client)
bucket = sagemaker_session.default_bucket()
print(f"Bucket: {bucket}")

Region: us-east-1
Bucket: sagemaker-us-east-1-875692608981


In [9]:
from datetime import datetime
from sagemaker.lineage.context import Context
from sagemaker.lineage.action import Action
from sagemaker.lineage.association import Association
from sagemaker.lineage.artifact import Artifact

unique_id = str(int(datetime.now().replace(microsecond=0).timestamp()))

print(f"Unique id is {unique_id}")

Unique id is 1637114413


## Cleanup

In [10]:
import time
def delete_associations(arn):
    # delete incoming associations
    incoming_associations = Association.list(destination_arn=arn)
    for summary in incoming_associations:
        assct = Association(
            source_arn=summary.source_arn,
            destination_arn=summary.destination_arn,
            sagemaker_session=sagemaker_session,
        )
        assct.delete()
        

    # delete outgoing associations
    outgoing_associations = Association.list(source_arn=arn)
    for summary in outgoing_associations:
        assct = Association(
            source_arn=summary.source_arn,
            destination_arn=summary.destination_arn,
            sagemaker_session=sagemaker_session,
        )
        assct.delete()


def delete_lineage_data():
    for summary in Context.list():
        print(f"Deleting context {summary.context_name}")
        delete_associations(summary.context_arn)
        ctx = Context(context_name=summary.context_name, sagemaker_session=sagemaker_session)
        ctx.delete()
        time.sleep(1)

    for summary in Action.list():
        print(f"Deleting action {summary.action_name}")
        delete_associations(summary.action_arn)
        actn = Action(action_name=summary.action_name, sagemaker_session=sagemaker_session)
        actn.delete()
        time.sleep(1)

    for summary in Artifact.list():
        print(f"Deleting artifact {summary.artifact_arn} {summary.artifact_name}")
        delete_associations(summary.artifact_arn)
        artfct = Artifact(artifact_arn=summary.artifact_arn, sagemaker_session=sagemaker_session)
        artfct.delete()
        time.sleep(1)


delete_lineage_data()